In [1]:
import pandas
pandas.options.display.float_format = '{:,.3f}'.format
tickers = [
    "WETH/USDC/0pt05",
    # "WBTC/USDC/0pt3",
    # "UNI/USDC/0pt3",
    # "LINK/USDC/0pt3",
    # "MATIC/USDC/0pt3",
    ]

# End of initial settings

In [2]:
# Starting back test review
Agg_Baseline_Early=pandas.read_pickle('https://newfrontiercm.com/Agg_Baseline_Public.pkl' ) 

In [3]:
# Signal frequency
import calendar
tbl1=Agg_Baseline_Early.loc[(Agg_Baseline_Early['Target']==0.05)&(Agg_Baseline_Early['Stop']==0.05),['Symbol','Date','Target','Stop']]
tbl1['Year']=tbl1['Date'].dt.year
tbl1['Month']=tbl1['Date'].dt.month
# tbl1['Month'] = tbl1['Month'].apply(lambda x: calendar.month_abbr[x])
tbl1.pivot_table(
    index=[
        'Symbol',
        "Year",    
    ],
    columns=["Month"],
    values=["Date"],
    aggfunc=["count", ],
    fill_value=0,
    observed=True,margins=True,
    sort=False,
)

count                                             
                      Date                                             
Month                    1   2   3   4   6   7   8   9  10  11  12  All
Symbol          Year                                                   
WETH/USDC/0pt05 2022     0   0   0   0  18  24  28  36  19  29  34  188
                2023    27  28  31  20   0   0   0   0   0   0   0  106
All                     27  28  31  20  18  24  28  36  19  29  34  294

In [4]:
#Order type
tbl1=Agg_Baseline_Early.loc[(Agg_Baseline_Early['Target']==0.05)&(Agg_Baseline_Early['Stop']==0.05),['Symbol','Date','Target','Stop','Long_OrderType',]]
tbl1.pivot_table(
    index=[
        "Symbol",    
    ],
    columns=["Long_OrderType"],
    values=["Date"],
    aggfunc=["count", ],
    fill_value=0,
    observed=True,margins=True,
    sort=False,
)

count          
                 Date          
Long_OrderType  LIMIT STOP  All
Symbol                         
WETH/USDC/0pt05   129  165  294
All               129  165  294

In [5]:
# Win_Loss by Order_Type

tbl1=Agg_Baseline_Early.loc[(Agg_Baseline_Early['Target']==0.05)&(Agg_Baseline_Early['Stop']==0.05),['Symbol','Date','Target','Stop','Long_OrderType','T1_WHResult_ExActive','T1_Direction',]]
tbl1.pivot_table(
    index=[
        "Symbol",   
        "Long_OrderType" ,
    ],
    columns=["T1_WHResult_ExActive"],
    values=["Date"],
    aggfunc=["count", ],
    fill_value=0,
    observed=True,margins=True,
    sort=False,
)

count                                
                                        Date                                
T1_WHResult_ExActive           LOSS-FULL-STP WIN-AMBIGUOUS WIN-FULL-TGT  All
Symbol          Long_OrderType                                              
WETH/USDC/0pt05 STOP                      79             6           80  165
                LIMIT                     69             4           56  129
All                                      148            10          136  294

In [6]:
# Win_Loss by Random
tbl1=Agg_Baseline_Early.loc[(Agg_Baseline_Early['Target']==0.05)&(Agg_Baseline_Early['Stop']==0.05),['Symbol','Date','Target','Stop','T1_WHResult_ExActive','RandSampleClass']]
tbl1.pivot_table(
    index=[
        "Symbol",   
        'RandSampleClass',
    ],
    columns=["T1_WHResult_ExActive"],
    values=["Date"],
    aggfunc=["count", ],
    fill_value=0,
    observed=True,margins=True,
    sort=False,
)

count                                
                                         Date                                
T1_WHResult_ExActive            LOSS-FULL-STP WIN-AMBIGUOUS WIN-FULL-TGT  All
Symbol          RandSampleClass                                              
WETH/USDC/0pt05 RSample1                   31             1           33   65
                RSample2                   29             1           24   54
                RSample3                   32             2           15   49
                RSample4                   24             4           34   62
                RSample5                   32             2           30   64
All                                       148            10          136  294

In [7]:
# Frequency by Entry Time
tbl1=Agg_Baseline_Early.loc[(Agg_Baseline_Early['Target']==0.05)&(Agg_Baseline_Early['Stop']==0.05),['Symbol','Date','Target','Stop','Long_OrderType','T1_TimeDeltaClass',]]
tbl1.pivot_table(
    index=[
        "Symbol",    
        'Long_OrderType',
    ],
    columns=["T1_TimeDeltaClass"],
    values=["Date"],
    aggfunc=["count", ],
    fill_value=0,
    observed=True,margins=True,
    sort=False,
)

count                                  
                                Date                                  
T1_TimeDeltaClass               0Day 0DayO/N 1Day 4Day +4Day NOTx  All
Symbol          Long_OrderType                                        
WETH/USDC/0pt05 STOP             158       1    3    1     1    1  165
                LIMIT            125       3    1    0     0    0  129
All                              283       4    4    1     1    1  294

In [8]:
# Sensitivity
def highlight_cells(val):
    color = 'YELLOW' if val > 0.5 else ''
    return 'background-color: {}'.format(color)

tbl1=Agg_Baseline_Early.loc[
    (Agg_Baseline_Early["Target"].between(0.040, 0.060))
    & (Agg_Baseline_Early["Stop"].between(0.040, 0.060)),
    ["Symbol", "Date", 'Target','Stop', "T1_WHResult_ExActive",'T1_WHExitPNL_ExActive','Long_OrderType', 'T1_TimeDeltaClass'],
].pivot_table(
    index=[
        "Symbol",
        'Target',
        'Stop',
        
    ],
    columns=["T1_WHResult_ExActive"],
    values=["T1_WHExitPNL_ExActive"],
    aggfunc=["count", ],
    observed=True,
)
tbl1.columns = ['_'.join(col) for col in tbl1.columns.values]
tbl1=tbl1.reset_index()
tbl1['Win_%']=round(tbl1['count_T1_WHExitPNL_ExActive_WIN-FULL-TGT']/(tbl1['count_T1_WHExitPNL_ExActive_LOSS-FULL-STP']+tbl1['count_T1_WHExitPNL_ExActive_WIN-AMBIGUOUS']+tbl1['count_T1_WHExitPNL_ExActive_WIN-FULL-TGT']),4)

tbl1.drop(columns=['count_T1_WHExitPNL_ExActive_LOSS-FULL-STP',
 'count_T1_WHExitPNL_ExActive_WIN-AMBIGUOUS',
 'count_T1_WHExitPNL_ExActive_WIN-FULL-TGT',]).pivot_table(
    index=[
    'Symbol','Stop',   
    ],
    columns=['Target',],
    values=["Win_%"],
    aggfunc=["sum", ],
    observed=True,
).style.applymap(
    highlight_cells
)